# Create input files for MaGeCK

### Load libraries

In [ ]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(rasilabRtemplates))

In [ ]:
args <- commandArgs(trailingOnly = TRUE)
treatment_counts_file <- args[1]
control_counts_file <- args[2]
insert_annotation_file <- args[3]
umi_cutoff <- as.numeric(args[4])
output_file <- args[5]
# treatment_counts_file <- "../data/insert_counts/219p14.csv"
# control_counts_file <- "../data/insert_counts/219p3.csv"
# insert_annotation_file <- "../../20221229_exp182_rbp_dual_sgrna_linkage/annotations/insert_annotations/rbp_dual_sgrna_linkage.csv"
# umi_cutoff <- 20

In [ ]:
insert_annotations <- read_csv(insert_annotation_file, show_col_types = FALSE) %>%
  select(insert_num, gene, sgrna)

In [ ]:
counts <- list(
"treatment" = read_csv(treatment_counts_file, show_col_types = FALSE),
"control" = read_csv(control_counts_file, show_col_types = FALSE)
)

bind_rows(counts, .id = "sample") %>%
  select(insert_num, barcode_group, sample, umi_count) %>%
  pivot_wider(names_from = sample, values_from = umi_count, values_fill = umi_cutoff) %>%
  left_join(insert_annotations, by = "insert_num") %>%
  mutate(gene = str_c(gene, barcode_group, sep = "_")) %>%
  mutate(sgrna = str_c(sgrna, barcode_group, sep = "_")) %>%
  select(sgrna, gene, treatment, control) %>%
  write_tsv(output_file)